In [1]:
from IPython.display import Image, display
import numpy as np
import os
from os.path import join
from PIL import ImageFile
import pandas as pd
from matplotlib import cm
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.mixture import GaussianMixture
from sklearn.isotonic import IsotonicRegression
import re

ImageFile.LOAD_TRUNCATED_IMAGES = True
plt.style.use('fivethirtyeight')
%matplotlib inline

In [22]:
train_img_dir_n = "C:\\Github\\Image-Processing-Project\\Dataset\\natural_images\\flower"
train_img_paths_n = [join(train_img_dir_n,filename) for filename in os.listdir(train_img_dir_n)]

In [3]:
train_img_paths, test_img_paths_flower = train_test_split(train_img_paths_n, test_size=0.25, random_state=42)
train_img_paths, val_img_paths_flower = train_test_split(train_img_paths, test_size=0.25, random_state=42)

In [4]:
natural_images_path = "C:\\Github\\Image-Processing-Project\\Dataset\\natural_images\\"
test_img_paths_no_flower = []
for d in [d for d in os.listdir("C:\\Github\\Image-Processing-Project\\Dataset\\natural_images") if d!= "flower"]:
    test_img_dir_na = natural_images_path+d
    test_img_paths_no_flower.append([join(test_img_dir_na,filename) for filename in os.listdir(test_img_dir_na)])
    
test_img_paths_no_flower_flat = [item for sublist in test_img_paths_no_flower for item in sublist]
test_img_paths_no_flower, val_img_paths_no_flower = train_test_split(test_img_paths_no_flower_flat, test_size = 0.25, random_state = 42)

In [41]:
def natural_img_dir(image_path):
    path_regex = r'natural_images\\(W*)'
    if 'natural_images' in image_path:
        return re.findall(path_regex,image_path,re.MULTILINE)[0].strip()
    else:
        return 'flower'

In [42]:
all_test_paths = test_img_paths_flower+test_img_paths_no_flower
test_path_df = pd.DataFrame({'path': all_test_paths,'is_flower': [1 if path in test_img_paths_flower else 0 
                                                                  for path in all_test_paths]})
test_path_df = shuffle(test_path_df,random_state = 0).reset_index(drop = True)
test_path_df['image_type'] = test_path_df['path'].apply(lambda x: natural_img_dir(x))
all_test_paths = test_path_df['path'].tolist()

In [43]:
all_val_paths = val_img_paths_flower+val_img_paths_no_flower
val_path_df = pd.DataFrame({
    'path': all_val_paths,
    'is_flower': [1 if path in val_img_paths_flower else 0 for path in all_val_paths]
})
val_path_df = shuffle(val_path_df,random_state = 0).reset_index(drop = True)
val_path_df['image_type'] = val_path_df['path'].apply(lambda x: natural_img_dir(x))
all_val_paths = val_path_df['path'].tolist()

In [44]:
image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    #output = img_array
    output = preprocess_input(img_array)
    return(output)

X_train = read_and_prep_images(train_img_paths)
X_test = read_and_prep_images(all_test_paths)
X_val = read_and_prep_images(all_val_paths)

In [49]:
X_train.shape

(474, 224, 224, 3)

In [47]:
resnet_weights_path = 'C://Github//Image-Processing-Project//ResNet50_weight//resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# X : images numpy array
resnet_model = ResNet50(input_shape=(image_size, image_size, 3), weights=resnet_weights_path, include_top=False, pooling='avg')  # Since top layer is the fc layer used for predictions

X_train = resnet_model.predict(X_train)
X_test = resnet_model.predict(X_test)
X_val = resnet_model.predict(X_val)

ValueError: Cannot assign value to variable ' conv3_block1_0_conv/kernel:0': Shape mismatch.The variable shape (1, 1, 256, 512), and the assigned value shape (512, 128, 1, 1) are incompatible.